In [8]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

!pip install optuna -q

import pickle
import pandas as pd
import numpy as np
import optuna
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. 데이터 로드

In [9]:
print("="*60)
print("📂 데이터 로드")
print("="*60)

LOAD_PATH = '/content/drive/MyDrive/preprocessed_data.pkl'

with open(LOAD_PATH, 'rb') as f:
    data_dict = pickle.load(f)

X_train = data_dict['X_train_clf']
X_val = data_dict['X_val_clf']
X_test = data_dict['X_test_clf']
y_train = data_dict['y_train_clf']
y_val = data_dict['y_val_clf']
y_test = data_dict['y_test_clf']

# Train + Val 합치기
X_train_full = pd.concat([X_train, X_val], axis=0).reset_index(drop=True)
y_train_full = pd.concat([y_train, y_val], axis=0).reset_index(drop=True)

print(f"✅ Train+Val: {X_train_full.shape}, Test: {X_test.shape}")

📂 데이터 로드
✅ Train+Val: (87026, 45), Test: (9670, 45)


In [10]:
# =============================================================================
# 스케일링
# =============================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

In [11]:
# =============================================================================
# GPU 설정
# =============================================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Device: {device}")

✅ Device: cuda


# 2. Ft-Transformer: 기본

## 2.1. Ft-Transformer 정의

In [15]:
class FTTransformer(nn.Module):
    def __init__(self, input_dim, num_classes, embed_dim=64, num_heads=4,
                 num_layers=3, ff_dim=128, dropout=0.1):
        super().__init__()
        self.feature_embedding = nn.Linear(1, embed_dim)
        self.input_dim = input_dim
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1, input_dim + 1, embed_dim))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(embed_dim)
        self.fc = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, num_classes)
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = x.unsqueeze(-1)
        x = self.feature_embedding(x)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)
        x = x + self.pos_embedding
        x = self.transformer(x)
        cls_output = x[:, 0]
        cls_output = self.norm(cls_output)
        output = self.fc(cls_output)
        return output

In [16]:
# =============================================================================
# 학습 함수
# =============================================================================
def train_and_evaluate(params, X_tr, y_tr, X_val, y_val, epochs=30, verbose=False):
    """모델 학습 및 평가"""

    X_tr_t = torch.FloatTensor(X_tr).to(device)
    y_tr_t = torch.LongTensor(y_tr).to(device)
    X_val_t = torch.FloatTensor(X_val).to(device)
    y_val_t = torch.LongTensor(y_val).to(device)

    train_loader = DataLoader(TensorDataset(X_tr_t, y_tr_t), batch_size=params['batch_size'], shuffle=True)

    model = FTTransformer(
        input_dim=X_tr.shape[1],
        num_classes=3,
        embed_dim=params['embed_dim'],
        num_heads=params['num_heads'],
        num_layers=params['num_layers'],
        ff_dim=params['ff_dim'],
        dropout=params['dropout']
    ).to(device)

    criterion = nn.CrossEntropyLoss(label_smoothing=params.get('label_smoothing', 0.0))
    optimizer = torch.optim.AdamW(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)

    best_val_acc = 0
    best_model_state = None
    patience_counter = 0

    for epoch in range(epochs):
        # Train
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_t)
            val_pred = torch.argmax(val_outputs, dim=1)
            val_acc = (val_pred == y_val_t).float().mean().item()

        if verbose and (epoch + 1) % 10 == 0:
            print(f"   Epoch {epoch+1}: Val Acc = {val_acc:.4f}")

        # Early Stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= 5:
                if verbose:
                    print(f"   Early stopping at epoch {epoch+1}")
                break

    model.load_state_dict(best_model_state)
    return model, best_val_acc

## 2.2. Ft-Transformer 학습 및 평가

In [19]:
print("\n" + "="*60)
print("🤖 FT Transformer (기본 버전)")
print("="*60)

base_params = {
    'embed_dim': 64,
    'num_heads': 4,
    'num_layers': 3,
    'ff_dim': 128,
    'dropout': 0.1,
    'lr': 1e-3,
    'weight_decay': 1e-4,
    'batch_size': 1024,
    'label_smoothing': 0.0
}

base_model, _ = train_and_evaluate(
    base_params,
    X_train_scaled, y_train.values,
    X_test_scaled, y_test.values,
    epochs=30, verbose=True
)

X_test_t = torch.FloatTensor(X_test_scaled).to(device)
base_model.eval()
with torch.no_grad():
    base_pred = torch.argmax(base_model(X_test_t), dim=1).cpu().numpy()

base_acc = accuracy_score(y_test, base_pred)
base_f1 = f1_score(y_test, base_pred, average='macro')

print(f"\nAccuracy: {base_acc:.4f}")
print(f"F1 Score (macro): {base_f1:.4f}")


🤖 FT Transformer (기본 버전)
   Epoch 10: Val Acc = 0.7030
   Epoch 20: Val Acc = 0.7058
   Early stopping at epoch 24

Accuracy: 0.7073
F1 Score (macro): 0.6818


# 3. Ft-Transformer: 최적화

## 3.1. Optuna 최적화

In [22]:
print("\n" + "="*60)
print("🤖 FT Transformer (Optuna 최적화)")
print("="*60)

def objective_ft(trial):
    params = {
        'embed_dim': trial.suggest_categorical('embed_dim', [32, 64]),
        'num_heads': trial.suggest_categorical('num_heads', [4, 8]),
        'num_layers': trial.suggest_int('num_layers', 2, 4),
        'ff_dim': trial.suggest_categorical('ff_dim', [64, 128]),
        'dropout': trial.suggest_float('dropout', 0.1, 0.3),
        'lr': trial.suggest_float('lr', 1e-4, 1e-3, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [512, 1024]),
        'label_smoothing': trial.suggest_float('label_smoothing', 0.0, 0.15)
    }

    if params['embed_dim'] % params['num_heads'] != 0:
        return 0.0

    X_tr, X_val_split, y_tr, y_val_split = train_test_split(
        X_train_scaled, y_train.values,
        test_size=0.2, random_state=42, stratify=y_train
    )

    _, best_acc = train_and_evaluate(params, X_tr, y_tr, X_val_split, y_val_split, epochs=20)
    return best_acc

optuna.logging.set_verbosity(optuna.logging.WARNING)
study_ft = optuna.create_study(direction='maximize')
study_ft.optimize(objective_ft, n_trials=15, show_progress_bar=True)

print(f"\n✅ 최적 파라미터: {study_ft.best_params}")


🤖 FT Transformer (Optuna 최적화)


  0%|          | 0/15 [00:00<?, ?it/s]


✅ 최적 파라미터: {'embed_dim': 64, 'num_heads': 4, 'num_layers': 4, 'ff_dim': 64, 'dropout': 0.10866552753221051, 'lr': 0.0009971511157826653, 'weight_decay': 2.5112890083621004e-05, 'batch_size': 1024, 'label_smoothing': 0.11744100649707297}


## 3.2. Ft-Transformer 학습 및 성능 평가

In [25]:
opt_model, _ = train_and_evaluate(
    study_ft.best_params,
    X_train_scaled, y_train.values,
    X_test_scaled, y_test.values,
    epochs=50, verbose=True
)

opt_model.eval()
with torch.no_grad():
    opt_pred = torch.argmax(opt_model(X_test_t), dim=1).cpu().numpy()

opt_acc = accuracy_score(y_test, opt_pred)
opt_f1 = f1_score(y_test, opt_pred, average='macro')

print(f"\nAccuracy: {opt_acc:.4f}")
print(f"F1 Score (macro): {opt_f1:.4f}")

   Epoch 10: Val Acc = 0.7030
   Epoch 20: Val Acc = 0.7125
   Epoch 30: Val Acc = 0.7270
   Epoch 40: Val Acc = 0.7310
   Epoch 50: Val Acc = 0.7347

Accuracy: 0.7347
F1 Score (macro): 0.7179


In [26]:
# =============================================================================
# 비교 결과
# =============================================================================
print("\n" + "="*60)
print("📊 기본 vs 최적화 비교")
print("="*60)

comparison = pd.DataFrame({
    'Model': ['FT Transformer (기본)', 'FT Transformer (Optuna)'],
    'Accuracy': [base_acc, opt_acc],
    'F1 (macro)': [base_f1, opt_f1]
})
print(comparison.to_string(index=False))


📊 기본 vs 최적화 비교
                  Model  Accuracy  F1 (macro)
    FT Transformer (기본)  0.707342    0.681775
FT Transformer (Optuna)  0.734747    0.717948


In [27]:
SAVE_PATH = '/content/drive/MyDrive/best_ft_model.pkl'

best_model = opt_model if opt_f1 > base_f1 else base_model
best_params = study_ft.best_params if opt_f1 > base_f1 else base_params

save_dict = {
    'model_state': best_model.state_dict(),
    'best_params': best_params,
    'scaler': scaler,
    'input_dim': X_train_full.shape[1],
    'feature_columns': X_train_full.columns.tolist()
}

with open(SAVE_PATH, 'wb') as f:
    pickle.dump(save_dict, f)

print(f"\n✅ 모델 저장 완료: {SAVE_PATH}")


✅ 모델 저장 완료: /content/drive/MyDrive/best_ft_model.pkl
